In [3]:
##### Data Type No.2: Naver SearchAd #####
    #a. 네이버 키워드 일일검색량 (ex. 네이버_키워드_일일검색량_2019-07-04.csv)

import pandas as pd
import numpy as np
import time
import os
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from datetime import date, datetime, timedelta
from pytz import timezone

try:
    os.remove("/Users/danie1ryu/Downloads/datalab.xlsx")
except FileNotFoundError: 
    pass

file_download_path = '/Users/danie1ryu/Downloads/'
file_save_path = '/Users/danie1ryu/Downloads/'
chromedriver_path = '/Users/danie1ryu/Desktop/chromedriver'

naver_id = 'onrikorea'
naver_pw = 'onrikorea00!'
keyword = input("검색어를 입력해주세요: ")

    
today = datetime.now(timezone('Asia/Seoul'))
yesterday = today - timedelta(1)
yesterday_period = '.'.join(yesterday.strftime("%Y-%m-%d %H:%M:%S").split(' ')[0].split('-'))
yesterday_hyphen = yesterday.strftime("%Y-%m-%d %H:%M:%S").split(' ')[0]

browser = webdriver.Chrome(chromedriver_path)
browser.get('https://datalab.naver.com/keyword/trendSearch.naver')
browser.find_element_by_xpath('//*[@id="lnb"]/div/ul/li[3]/a').send_keys(Keys.RETURN)
browser.find_element_by_id('item_keyword1').send_keys(keyword)

time.sleep(4)

browser.find_element_by_xpath('//*[@id="content"]/div/div[2]/div[1]/div/form/fieldset/div/div[6]/div[1]/label[2]').click()

time.sleep(3)

#모바일 & PC 전체 검색량 옵션 클릭
browser.find_element_by_xpath('//*[@id="item_device_0"]').click()

time.sleep(3)
browser.find_element_by_xpath('//*[@id="content"]/div/div[2]/div[1]/div/form/fieldset/a/span').click()

time.sleep(5)

#엑셀 파일 다운로드
browser.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[1]/div/div/div/div/div/div[1]/div[4]/a').send_keys(Keys.RETURN)

time.sleep(3)

#browser.close()  

browser.get('https://searchad.naver.com/login?returnUrl=https:%2F%2Fmanage.searchad.naver.com%2Fcustomers%2F1519222%2Ftool%2Fkeyword-planner')
browser.find_element_by_id('uid').send_keys(naver_id)
browser.find_element_by_id('upw').send_keys(naver_pw)
browser.find_element_by_xpath('//*[@id="container"]/div/div/fieldset/div/span/button').send_keys(Keys.RETURN)

browser.get('https://manage.searchad.naver.com/customers/1519222/tool/keyword-planner?keywords=')

browser.find_element_by_xpath('/html/body/elena-root/elena-wrap/div/div[2]/elena-tool-wrap/div/div/div/div/elena-keyword-planner/div[2]/div[1]/div[1]/div[2]/div/div/div[1]/div/textarea').send_keys(keyword)
browser.find_element_by_xpath('/html/body/elena-root/elena-wrap/div/div[2]/elena-tool-wrap/div/div/div/div/elena-keyword-planner/div[2]/div[1]/div[1]/div[3]/button').send_keys(Keys.RETURN)

time.sleep(5)

try:
    PC_monthly_search_traffic = int(''.join(browser.find_element_by_css_selector('#wgt-{} > td.elenaColumn-monthlyPcQcCnt > div > span'.format(keyword)).text.split(',')))
except ValueError:
    PC_monthly_search_traffic = 0
    

try:
    Mobile_monthly_search_traffic = int(''.join(browser.find_element_by_css_selector('#wgt-{} > td.elenaColumn-monthlyMobileQcCnt > div > span'.format(keyword)).text.split(',')))
except ValueError:
    Mobile_monthly_search_traffic = 0
    
    
total_monthly_search_traffic = PC_monthly_search_traffic + Mobile_monthly_search_traffic

browser.close()

df_total = pd.read_excel(file_download_path + 'datalab.xlsx', header=None).loc[7:]

df_total['키워드'] = keyword

#df_total['일일검색량'] = ''

df_total.rename(columns={0:'날짜',
                      1:'Proportion(%)'}, 
                 inplace=True)

df_total.loc[7:, 'Proportion(%)'] = pd.to_numeric(df_total.loc[7:, 'Proportion(%)'])

total_prop_total = df_total['Proportion(%)'].sum()
df_total['일일검색량'] = df_total['Proportion(%)'] / total_prop_total * total_monthly_search_traffic


df_total['일일검색량'] = df_total['일일검색량'].astype(int)


df_total = df_total.drop('Proportion(%)', axis=1)

df_total = df_total.set_index('날짜')

#df_total.to_csv(file_save_path + keyword + '_키워드검색량_' + yesterday_hyphen + '.csv', header=True)
df_total.to_excel(file_save_path + keyword + '_키워드검색량_' + yesterday_hyphen + '.xlsx', header=True)
                

검색어를 입력해주세요: 클라뷰여배우크림
